First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [7]:
ltw = pd.read_csv(f'{path}/data/ltw_treat.csv', encoding = 'UTF-8-SIG', sep=',')
ltw.head(10)

,AGS,GEN,year,Land,state_ID,Kreis,post_2010,post_2014,Union,SPD,...,FDP,Linke,Andere,treatment,treated_0,treated_10,treated_20,treated_30,treated_50,treated_100
0,3101000,Braunschweig,2008.0,NI,3,3101,0.0,0.0,36.861503,31.245464,...,8.087549,9.549529,3.232192,100,0.0,0.0,0.0,0.0,0.0,1.0
1,3101000,Braunschweig,2013.0,NI,3,3101,1.0,0.0,29.666789,33.671272,...,8.250600,4.246117,5.556723,100,0.0,0.0,0.0,0.0,0.0,1.0
2,3101000,Braunschweig,2017.0,NI,3,3101,1.0,1.0,26.342641,37.604703,...,7.904115,6.741527,9.544639,100,0.0,0.0,0.0,0.0,0.0,1.0
3,3102000,Salzgitter,2008.0,NI,3,3102,0.0,0.0,37.476063,39.268958,...,5.740138,8.828035,4.394868,100,0.0,0.0,0.0,0.0,0.0,1.0
4,3102000,Salzgitter,2013.0,NI,3,3102,1.0,0.0,30.880127,44.075623,...,6.053921,4.135136,5.602442,100,0.0,0.0,0.0,0.0,0.0,1.0
5,3102000,Salzgitter,2017.0,NI,3,3102,1.0,1.0,24.445995,45.203436,...,5.319891,4.862732,15.901927,100,0.0,0.0,0.0,0.0,0.0,1.0
6,3103000,Wolfsburg,2008.0,NI,3,3103,0.0,0.0,43.121141,31.515784,...,6.691886,7.863727,4.298200,None,0.0,0.0,0.0,0.0,0.0,0.0
7,3103000,Wolfsburg,2013.0,NI,3,3103,1.0,0.0,37.629895,35.812910,...,7.342017,3.210075,5.103196,None,0.0,0.0,0.0,0.0,0.0,0.0
8,3103000,Wolfsburg,2017.0,NI,3,3103,1.0,1.0,29.942107,43.211115,...,6.092242,4.525280,10.474720,None,0.0,0.0,0.0,0.0,0.0,0.0
9,3151001,Adenbüttel,2008.0,NI,3,3151,0.0,0.0,47.568209,30.960854,...,7.117438,4.863582,3.202847,100,0.0,0.0,0.0,0.0,0.0,1.0


Statsmodels

Define a function for regression models

In [ ]:
# call: regression(post, dataset, 'cluster_level', controls='', fe='')
def regression(post, dataset, cluster_level, **kwargs):
    tables = []
    results = {}
    treatment_dummies = ['treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100']
    parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
    post_dummies = {'post_2010': 'after 2010', 'post_2014': 'in 2014'}
    cluster_levels = {'AGS': 'Municipality', 'Kreis': 'County', 'state_id': 'State'}
    n_clust = len(dataset[cluster_level].unique())
    for treatment in treatment_dummies:
        for party in parties:
            if 'controls' in kwargs:
                controls = kwargs['controls']
                results[party] = smf.ols(f'{party} ~ {treatment}*{post} + {controls}', data=dataset).fit(cov_type='cluster', cov_kwds={'groups': np.array(dataset[cluster_level])}, missing='drop')
            elif 'fe' in kwargs:
                fe = kwargs['fe']
                results[party] = smf.ols(f'{party} ~ {treatment}:{post} + {fe}', data=dataset).fit(cov_type='cluster', cov_kwds={'groups': np.array(dataset[cluster_level])}, missing='drop')
            else:
                results[party] = smf.ols(f'{party} ~ {treatment}*{post}', data=dataset).fit(cov_type='cluster', cov_kwds={'groups': np.array(dataset[cluster_level])}, missing='drop')
        stargazer = Stargazer([results['Union'], results['SPD'], results['FDP'], results['Linke'], results['Grüne'], results['Andere']])
        distance = treatment.strip('treated_')
        #stargazer.title(f'Effect on Primary Vote, Municipalities within {distance} km of a powerline project, treatment between 2013 and 2017')
        stargazer.title(f'Effect on Vote Shares, treatment {post_dummies[post]}. Standard errors adjusted for {n_clust} clusters on the {cluster_levels[cluster_level]} level')
        stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
        stargazer.significant_digits(3)
        if 'fe' not in kwargs:
            stargazer.covariate_order([treatment, post, f'{treatment}:{post}'])
        elif 'fe' in kwargs:
            stargazer.covariate_order([f'{treatment}:{post}'])
        #stargazer.rename_covariates(covar_names)
        stargazer.show_model_numbers(False)
        #stargazer.add_custom_notes(['First note', 'Second note'])
        if 'fe' in kwargs:
            if 'state' in fe:
                stargazer.add_line('State FE', ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', ])
            if 'year' in fe:
                stargazer.add_line('Year FE', ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', ])
        table = stargazer.render_latex()
        tables.append(table)
    return tables

Define a function to export tables in one .tex dok

In [ ]:
def tab_to_tex(name, tables):
    table_parts = []
    with open(f'{path}/tables/ltw/{name}.tex', 'w') as f:
        header = r'\documentclass[12pt]{article}' + '\n' + r'\usepackage[a1paper]{geometry}' + '\n' + r'\begin{document}' + '\n \n'
        f.write(header)
        for index, table in enumerate(tables, start=1): 
            table = table.replace('t_', 't')
            table = table.replace('d_', 'd')
            table = table.replace('0:p', '0*p')
            lines = r'\hline \\[-1.8ex]'
            table_parts = table.split(lines)
            if index == 1:
                f.write(table_parts[0] + lines +  table_parts[1] + lines + table_parts[2] + lines)
            elif index < len(tables):
                f.write(table_parts[2] + lines)
            elif index == len(tables):
                f.write(table_parts[2] + lines + table_parts[3] + lines + table_parts[4])
        closer = '\n' + r'\end{document}'
        f.write(closer)

DiD for Treatment in 2014

In [ ]:
post = 'post_2014'
# primary vote
post_14 = regression(post, erst, 'Primary', 'AGS')
tab_to_tex('2017_erst_ags', post_14)